# HW 1- Logistic Regression
- 수업시간에 다룬 Logistic Regression Notebook을 참고로 하여 Diabetes데이터를 가지고 Logistic Regression으로 학습하고 테스트하는 모델을 작성
- 이 노트북 파일을 받아서 작성하고 화일명은 Hw1_학번 (예: Hw1_2020_12345, 반드시 하이픈대신 언더라인으로)으로 해서 etl에 탑재
- 아래의 지시사항대로 하나 필요할 경우 cell을 더 첨가해서 자유롭게 할 수 있다.
- 반드시 colab에서 실행되는지 확인 할 것.
- Due: 4월 2일 수업시간전까지

# Diabetes 데이터 셋의 구조(diabetes.csv)
![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-03-30%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2011.15.10.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-03-30%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2011.15.10.png)

- 8개의 자질과 맨 마지막 칼럼은 당뇨인지 아닌지, 0, 1로 되어 있음
- 총 768개의 행으로 되어 있음
- 읽을때 650개를 training 데이터로, 나머지를 test 데이터로 할 것

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd


## Step 1: Loading Diabetes Dataset
- 데이터 파일 읽어와서 train_x, train_y(정답), test_x, test_y(정답)로 구분함

In [3]:
diabetesDF = pd.read_csv('diabetes.csv')

train_x = torch.Tensor(diabetesDF.drop('Outcome',1)[:650].astype(float).values)
train_y = torch.Tensor(diabetesDF['Outcome'][:650].astype(float).values)
test_x = torch.Tensor(diabetesDF.drop('Outcome',1)[650:].astype(float).values)
test_y = torch.Tensor(diabetesDF['Outcome'][650:].astype(float).values)

In [5]:
diabetesDF

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [6]:
train_x

tensor([[6.0000e+00, 1.4800e+02, 7.2000e+01,  ..., 3.3600e+01, 6.2700e-01,
         5.0000e+01],
        [1.0000e+00, 8.5000e+01, 6.6000e+01,  ..., 2.6600e+01, 3.5100e-01,
         3.1000e+01],
        [8.0000e+00, 1.8300e+02, 6.4000e+01,  ..., 2.3300e+01, 6.7200e-01,
         3.2000e+01],
        ...,
        [0.0000e+00, 1.7900e+02, 5.0000e+01,  ..., 3.7800e+01, 4.5500e-01,
         2.2000e+01],
        [1.1000e+01, 1.3600e+02, 8.4000e+01,  ..., 2.8300e+01, 2.6000e-01,
         4.2000e+01],
        [0.0000e+00, 1.0700e+02, 6.0000e+01,  ..., 2.6400e+01, 1.3300e-01,
         2.3000e+01]])

In [1]:
# 정답은 0, 1로 되어 있어 float -> int로 
train_y = train_y.type(torch.int64)
test_y = test_y.type(torch.int64)

print(len(train_y))
print(type(train_y))

NameError: name 'train_y' is not defined

## STEP 2: MAKING DATASET ITERABLE

In [0]:
import torch.utils.data

In [0]:
# train_x, train_y를 텐서데이터 셋으로 변환

train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)

In [19]:
train_dataset[0][0].size()

torch.Size([8])

In [0]:
# Hyper-parameters 

batch_size = 10
n_iters = 300
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)


In [0]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)

In [0]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=False)

## STEP 3: CREATE MODEL CLASS

In [0]:
class LogisticRegressionModel(nn.Module):      #you have to define class
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

## STEP 4: INSTANTIATE MODEL CLASS

In [0]:
input_dim = 8   #You have to define based on the diabetes data
output_dim = 2  #You have to define based on the diabetes data

model = LogisticRegressionModel(input_dim, output_dim)


In [0]:
# Logistic regression model
model = #you have to define

#######################
###  USE GPU FOR MODEL  ##
#######################

## STEP 5: INSTANTIATE LOSS CLASS

In [0]:
criterion = nn.CrossEntropyLoss()

## STEP 6: INSTANTIATE OPTIMIZER CLASS

In [0]:
learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # you have to define


## STEP 7: TRAIN THE MODEL

In [36]:

#######################
#  USE GPU FOR MODEL  #
#######################

iter = 0

print(num_epochs)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        iter += 1
        
        if iter == 1:
            print(inputs, labels)
            print(len(inputs[0]))
        else:
            break


4
tensor([[1.0000e+00, 1.0000e+02, 6.6000e+01, 1.5000e+01, 5.6000e+01, 2.3600e+01,
         6.6600e-01, 2.6000e+01],
        [5.0000e+00, 1.0400e+02, 7.4000e+01, 0.0000e+00, 0.0000e+00, 2.8800e+01,
         1.5300e-01, 4.8000e+01],
        [3.0000e+00, 1.2900e+02, 9.2000e+01, 4.9000e+01, 1.5500e+02, 3.6400e+01,
         9.6800e-01, 3.2000e+01],
        [2.0000e+00, 9.1000e+01, 6.2000e+01, 0.0000e+00, 0.0000e+00, 2.7300e+01,
         5.2500e-01, 2.2000e+01],
        [2.0000e+00, 1.1000e+02, 7.4000e+01, 2.9000e+01, 1.2500e+02, 3.2400e+01,
         6.9800e-01, 2.7000e+01],
        [0.0000e+00, 1.0400e+02, 6.4000e+01, 2.3000e+01, 1.1600e+02, 2.7800e+01,
         4.5400e-01, 2.3000e+01],
        [2.0000e+00, 1.2300e+02, 4.8000e+01, 3.2000e+01, 1.6500e+02, 4.2100e+01,
         5.2000e-01, 2.6000e+01],
        [4.0000e+00, 1.3200e+02, 8.6000e+01, 3.1000e+01, 0.0000e+00, 2.8000e+01,
         4.1900e-01, 6.3000e+01],
        [1.0000e+01, 1.7900e+02, 7.0000e+01, 0.0000e+00, 0.0000e+00, 3.5100e+0

In [38]:
#######################
#  USE GPU FOR MODEL  #
#######################

iter = 0

print(num_epochs)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Load data as Variable
        inputs = inputs.requires_grad_()
        labels = labels
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output
        outputs = model(inputs)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 50 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0

            # Iterate through test dataset
            for inputs, labels in test_loader:
                # Load inputs to a Torch Variable
                inputs = inputs.requires_grad_()

                # Forward pass 
                outputs = model(inputs)

                # Get predictions
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


4
Iteration: 50. Loss: 23.38006591796875. Accuracy: 61
Iteration: 100. Loss: 3.3298702239990234. Accuracy: 62
Iteration: 150. Loss: 3.053244113922119. Accuracy: 50
Iteration: 200. Loss: 1.9589582681655884. Accuracy: 41
Iteration: 250. Loss: 11.328141212463379. Accuracy: 62


In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')